In [ ]:
print("Welcome to PathFinder: Press the Double Arrow above to begin! \nEnter a starting point, an end point, and any number of inhibited elements, then find your path.")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact_manual, interactive, GridspecLayout, AppLayout, Button, Layout, Widget
from IPython.display import clear_output
import numpy as np

In [ ]:
nodes = pd.read_excel("CClist.xls")

In [ ]:

nodes_a = set(nodes['Source'])
nodes_b = set(nodes['Target'])

#preliminary graph object
g = nx.from_pandas_edgelist(nodes, source = 'Source', target = 'Target', create_using =nx.DiGraph())

In [ ]:
## Functions
def on_go_button_clicked(go_button):
    
    inhiblist = list(inhib_dropdown.value)
#get start and end and inhib
    start = start_dropdown.value
    end = end_dropdown.value
    inhib = inhib_dropdown.value

# get the path 
    try:
        paths = list(nx.shortest_simple_paths(g, start, end))
        all_paths = []
        for path in paths:
            all_paths.extend(path)
    # Reset the graph with only active nodes and edges in the ID'd paths
        # The nodes
        G2 = nx.DiGraph()
        G2.add_nodes_from(all_paths)

        #the edges
        all_diads  = []
        for path in paths:
            start_num0 = 0
            start_num1 = 2

            path_len = len(path)
            connects = path_len-1

            for i in range(0,connects):
                path_diad = path[start_num0:start_num1]
                start_num0 += 1
                start_num1 += 1
                all_diads.append(path_diad)
        set_diads = set(tuple(x) for x in all_diads)


        # Add the edges to the graph
        G2.add_edges_from(set_diads)
        edges_list2=list(G2.edges)

        for i in range(0,len(edges_list2)):
            edges_list2[i]= list(edges_list2[i][0:2])


         # Assign colors
        edge_color_map = []
        for i in edges_list2:
            if i[0] in inhiblist:
                edge_color_map.append('lightgrey')
            else:
                edge_color_map.append('black')

        color_map = ['lightblue'] * len(G2.nodes)
        for i in range(0,len(G2.nodes)):
            if list(G2.nodes)[i] == start:
                color_map[i]='magenta'
            if list(G2.nodes)[i] == end:
                color_map[i]=('cyan')
            if list(G2.nodes)[i] in inhiblist:
                color_map[i]=('lightgrey')

        ##Graph the thing:
        pos = nx.layout.planar_layout(G2)
        #convert_to_directed(G2)
        nx.draw(G2,  
                pos,
                node_color=color_map, 
                with_labels=True, 
                arrowstyle="->",
                arrowsize=10,
                edge_color=edge_color_map,
               )

        real_count = 0
        label_count = 1
        
        for path in paths:
            check = set.intersection(set(path),set(inhiblist))
            if len(check) == 0:print("Path", label_count, ":", path)
            #if the member is in the path but not at the end
            if len(check) > 0:
                for inhib in check:
                    print('Path', label_count,  path, 'interrupted by inhibition of', inhib)
            real_count += 1
            label_count += 1
        with graph_out1:
            print("Paths from ", start, " to ", end)
            plt.show()
            

    except nx.NetworkXNoPath:
        with graph_out2:
            print("No Paths")
    except nx.NodeNotFound:
        with graph_out2:
            print("One of your selections is not in the database")
            


In [ ]:
from IPython.core.display import display, HTML
s = HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_above()"><input type="submit" id="toggleButton" value="Reset"></form>''')

start_options = list(g.nodes)
start_options.sort()

start_dropdown = widgets.Combobox(
    options=start_options,
    description='Start: ',
    placeholder='Type here',
    layout={'width': '50%'})

end_dropdown = widgets.Combobox(
    options=start_options,
    description='End: ',
    placeholder='Type here',
    layout={'width': '50%'})



inhib_dropdown = widgets.SelectMultiple(
    options=start_options,
    description='Inhibit: ',
    placeholder='Type here',
    layout={'width': '50%'})

##Create selection widgets
graph_out1 = widgets.Output()
graph_out2 = widgets.Output()

spacer = widgets.Label('')
instructions = widgets.Label('Choose Start, End and Inhibited elements')

go_button = widgets.Button(description='Produce Output',
                           button_style='',
                           display='flex',
                           flex_flow='column',
                           align_items='stretch')
go_button.on_click(on_go_button_clicked)


go_button.observe(start_dropdown.value)
go_button.observe(end_dropdown.value)
go_button.observe(inhib_dropdown.value)



widgets.VBox([instructions,start_dropdown,end_dropdown,inhib_dropdown,go_button, graph_out1,graph_out2])


In [ ]:
s